## Webscraping with Selenium and Chromedriver: www.eldiario.net

If the following packages are not installed, unhash the following lines of code and run them.

In [1]:
#!pip3 install webdriver-manager
#!pip3 install selenium

For this part, we will be working with a Chromedriver; however this could easily be change to a Firefoxdriver, depending on the browser of your preference.

In [2]:
# Required libraries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from uuid import uuid4
import re
import itertools
import functools
import operator
from datetime import date, timedelta
import pandas as pd
import json
import datetime
from tqdm import tqdm
import functions_pagina_siete as func         # .py where all the functions used are saved


Looking for [chromedriver 81.0.4044.69 mac64] driver in cache 
File found in cache by path [/Users/danielapintoveizaga/.wdm/drivers/chromedriver/81.0.4044.69/mac64/chromedriver]


### Steps

1. Defining daterange to extract data

In [3]:
start_dt = (date.today()-timedelta(days = 1))
end_dt = date.today()

dates = []
for dt in pd.date_range(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))

In [4]:
dates

['2020-05-01', '2020-05-02']

2. Defining urls to scrap

In [5]:
all_urls = []
for date in dates:
    year = re.split('-', date)[0]
    month = re.split('-', date)[1]
    day = re.split('-', date)[2]
    url = 'https://www.paginasiete.bo/archivo/' + year + '/' + month + '/' + day
    all_urls.append(url)

print('All urls we will be scraping:')
all_urls

All urls we will be scraping:


['https://www.paginasiete.bo/archivo/2020/05/01',
 'https://www.paginasiete.bo/archivo/2020/05/02']

3. We get a list of all links to the daterange determined

In [6]:
all_links_week = []
for url in all_urls:
    func.driver.get(url)
    
    links_page = []
    for s in range(len(func.driver.find_elements_by_class_name("archivo"))):
        func.driver_section = func.driver.find_elements_by_class_name("archivo")[s]
    
        len_links = len(func.driver_section.find_elements_by_class_name("listado-noticias")[0].find_elements_by_tag_name("li"))
        temp_link = []
        for l in range(len_links):
            link = func.driver_section.find_elements_by_class_name("listado-noticias")[0].find_elements_by_tag_name("li")[l].find_element_by_css_selector('a').get_attribute('href')
            temp_link.append(link)
        links_page.append(temp_link)
    all_links_week.append(links_page)

list_pages = functools.reduce(operator.iconcat, all_links_week, [])
flatten_list = functools.reduce(operator.iconcat, list_pages, [])


We extract de data required, such as:

+ id
+ title
+ author
+ newspaper
+ detail
+ date
+ date
+ section
+ body_article

In [7]:
articles_pagina_siete = {'week_x' : {}}
list_articles = []
for i in range(len(flatten_list)):
    func.driver.get(flatten_list[i])
    
    temp = {}
    temp['id'] = str(uuid4())
    temp['title'] = func.obtain_title(func.driver)
    temp['author'] = func.get_author(func.driver)
    temp['newspaper'] = 'pagina_siete'
    temp['detail'] = func.obtain_detail(func.driver)
    temp['date'] = func.get_date(flatten_list[i])
    temp['section'] = func.get_section(func.driver)
    temp['body_article'] = func.get_body(func.driver)
    
    list_articles.append(temp)
    #print('Finish ', i, '/', len(flatten_list))
articles_pagina_siete['week_1'] = list_articles

func.driver.close()  #closing driver


Check for author
no date found
Finish  0 / 111
Check for author
no date found
Finish  1 / 111
Check for author
no date found
Finish  2 / 111
Check for author
no date found
Finish  3 / 111
Check for author
no date found
Finish  4 / 111
Check for author
no date found
Finish  5 / 111
Check for author
no date found
Finish  6 / 111
Check for author
no date found
Finish  7 / 111
Check for author
no date found
Finish  8 / 111
Check for author
no date found
Finish  9 / 111
Check for author
no date found
Finish  10 / 111
Check for author
no date found
Finish  11 / 111
Check for author
no date found
Finish  12 / 111
Check for author
no date found
Finish  13 / 111
Check for author
no date found
Finish  14 / 111
Check for author
no date found
Finish  15 / 111
Check for author
no date found
Finish  16 / 111
Check for author
no date found
Finish  17 / 111
Check for author
no date found
Finish  18 / 111
Check for author
no date found
Finish  19 / 111
Check for author
no date found
Finish  20 / 111
Ch

### Viewing data in data frame format

4. Dataframe format using pandas

In [8]:
df = pd.DataFrame(data=list_articles, columns=['id', 'title', 'author', 'newspaper', 'detail', 'date', 'section', 'body_article'])
df


,id,title,author,newspaper,detail,date,section,body_article
0,e403d676-7816-4da4-bd5b-f02a9f0dae94,RECURSOS PARA LA GESTIÓN DE TALENTO HUMANO EN ...,mayor error,pagina_siete,,None,ECONOMÍA,"Incertidumbre, estrés y ansiedad, son algunas ..."
1,0e24f3f1-9a72-4f7b-8e29-6a33e84aea6d,BNB INSTALA AREAS DE DESINFECCIÓN EN AGENCIAS,mayor error,pagina_siete,,None,ECONOMÍA,El Banco Nacional de Bolivia (BNB) instaló c...
2,633e106b-5ded-4ace-bdbd-c0c59a40f31d,YPFB compensará baja nominación de Petrobras c...,mayor error,pagina_siete,La petrolera estatal venderá 4 MMmcd de gas a ...,None,ECONOMÍA,Página Siete Digital.El presidente de Yacimien...
3,45cbef16-b01d-4929-b1cc-0a52a7e8f30d,Cámara Nacional de Comercio apoya el “Plan Tra...,mayor error,pagina_siete,El gerente general de la Cámara Nacional de Co...,None,ECONOMÍA,Página Siete Digital / La Paz.El gerente de la...
4,8542db5c-50c7-405c-8986-62ef3c7ea1c3,Gobierno presentará recurso de inconstituciona...,mayor error,pagina_siete,La norma aprobada por el MAS establece un plaz...,None,NACIONAL,Página Siete Digital .El Gobierno anunció este...
5,1bb4c849-b2fe-4f1b-997e-a7bc99f67fc1,Añez anuncia plan de creación de 600 mil empleos,mayor error,pagina_siete,La Presidenta aseguró que se construirán hospi...,None,NACIONAL,Página Siete Digital.La presidenta Jeanine Añe...
6,3afb6f1e-d9d4-43a0-a306-912524892c2e,Mesa lamenta confrontación entre el MAS y Añez...,mayor error,pagina_siete,“Sería una vergüenza para el país que cuando m...,None,NACIONAL,Página Siete Digital/ La Paz.El candidato a la...
7,3e5b7b2d-f1f7-44a5-a266-d85844ab52d2,Congresista de EEUU: “ciber patrullas” de Boli...,mayor error,pagina_siete,,None,NACIONAL,Erbol.El congresista de Estados Unidos y presi...
8,af1f850e-1db9-400e-9a57-eb1b47318cfd,COB señala que Añez no atiende demandas de los...,mayor error,pagina_siete,"Ante el incumplimiento de demandas, y ante la ...",None,NACIONAL,Página Siete Digital / La Paz.La Central Obrer...
9,2944b4c4-8e22-463f-bd1a-daf80b36675c,Detienen a policía implicado en el tráfico de ...,mayor error,pagina_siete,Era sargento de la Felcn y fue arrestado con o...,None,SEGURIDAD,"Página Siete Digital.El ministro de Gobierno, ..."


### Saving output in csv

5. Saving output to .csv

In [9]:
df.to_csv('paginasiete.csv')